## 准备工作
### 创建图像数据、标注数据的文件路径列表

In [33]:
# 包装的import
import os.path as osp
import numpy as np
import cv2
import random

#从文件和文本中读取、加工、保存XML的库
import xml.etree.ElementTree as ET

import torch
import torch.utils.data as data

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 设定随机数的种子，
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [14]:
#创建学习、验证用图像数据和标注数据的文件路径列表

def make_datapath_list(rootpath):
    #rootpath:数据文件夹路径

    #创建图像文件和标注文件的路径模板
    imgpath_template = osp.join(rootpath,'JPEGImages','%s.jpg')
    annopath_template = osp.join(rootpath,'Annotations','%s.xml')

    #分别取得训练和验证用的文件的ID
    train_id_names = osp.join(rootpath+'ImageSets/Main/train.txt')
    val_id_names = osp.join(rootpath+'ImageSets/Main/val.txt')

    #创建训练数据的图像数据和标注文件的路径列表
    train_img_list = list()
    train_anno_list = list()

    for line in open(train_id_names):
        file_id = line.strip()                     #删除空格和换行符
        # print(file_id)
        img_path = (imgpath_template % file_id)    #图像的路径
        anno_path = (annopath_template % file_id)  #标注的路径
        train_img_list.append(img_path)            #添加到列表中
        train_anno_list.append(anno_path)          #添加到列表中

    #创建验证数据的图像数据和标注文件的路径列表
    val_img_list = list()
    val_anno_list = list()

    for line in open(val_id_names):
        file_id = line.strip()                     #删除空格和换行符
        # print(file_id)
        img_path = (imgpath_template % file_id)    #图像的路径
        anno_path = (annopath_template % file_id)  #标注的路径
        val_img_list.append(img_path)              #添加到列表中
        val_anno_list.append(anno_path)            #添加到列表中
    
    return train_img_list,train_anno_list,val_img_list,val_anno_list


In [15]:
rootpath = './data/VOCdevkit/VOC2012/'
train_img_list,train_anno_list,val_img_list,val_anno_list = make_datapath_list(rootpath)

print(train_img_list[0])
print(train_anno_list[0])

./data/VOCdevkit/VOC2012/JPEGImages\2008_000008.jpg
./data/VOCdevkit/VOC2012/Annotations\2008_000008.xml


### 将xml的标注数据转换成列表

In [47]:
# 将xml格式的标注转换成列表形式的类

class Anno_xml2list(object):
    def __init__(self,classes):
        self.classes = classes 
    
    def __call__(self,xml_path,width,height):
        """
        使用图像的尺寸信息，对每一张包含xml格式的标注数据进行正规化处理，并保存到列表中

        ret: [[xmin,ymin,xmax,ymax,label_ind],...]
        """

        #将图像包含的所有标注数据保存到列表中
        ret = []

        #读取xml文件
        xml = ET.parse(xml_path).getroot()

        #将图像内包含的物体数量作为循环次数进行迭代
        for obj in xml.iter('object'):

            #将标注中注明检测难度为difficult的对象剔除
            difficult = int(obj.find('difficult').text)
            if difficult == 1:
                # print(difficult)
                continue

            #用于保存每个物体的标注信息列表
            bndbox = []
            name = obj.find('name').text.lower().strip() #物体名称
            # print(name)
            bbox = obj.find('bndbox')
            
            #获取标注xmin、ymin、xmax、ymax,并归一化为0~1之间
            pts = ['xmin','ymin','xmax','ymax']
            for pt in pts:
                #VOC的原点是从（1，1）开始，减1变成（0，0）
                cur_pixl = int(bbox.find(pt).text) - 1

                #归一化
                if pt == 'xmin' or pt == 'xmax':
                    cur_pixl /= width
                else:
                    cur_pixl /= height
                bndbox.append(cur_pixl)
            #取得分类名的index并添加
            label_index = self.classes.index(name)
            bndbox.append(label_index)

            ret += [bndbox]

        return np.array(ret)


In [48]:
#确认
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
transform_anno = Anno_xml2list(voc_classes)

# 使用OpenCV读取图像
ind = 1
image_file_path = val_img_list[ind]
img = cv2.imread(image_file_path)   #(333, 500, 3) (height,width,RGB)
height,width,channels = img.shape 

#以列表形式表示标注
transform_anno(val_anno_list[ind],width,height)

array([[ 0.09      ,  0.03003003,  0.998     ,  0.996997  , 18.        ],
       [ 0.122     ,  0.56756757,  0.164     ,  0.72672673, 14.        ]])